In [6]:
import numpy as np
import os
from os.path import isfile
#from PIL import Image as Img
from pianition.data_util import *

import keras
from keras.models import Sequential, Model
from keras.utils import plot_model
from keras.layers import Input, Dense, TimeDistributed, LSTM, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, Flatten, Conv2D, BatchNormalization, Lambda
from keras.layers.advanced_activations import ELU
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras import backend
from keras.utils import np_utils
from keras.optimizers import Adam, RMSprop
from keras import regularizers

import librosa
import librosa.display
import matplotlib.pyplot as plt

import tensorflow as tf
tf.enable_eager_execution()

ModuleNotFoundError: No module named 'pianition'

In [3]:
SHUFFLE_BUFFER = 10
BATCH_SIZE = 32
NUM_CLASSES = 41

# Create a description of the features.  
feature_description = {
    'feature0': tf.FixedLenFeature([32768], tf.float32),
    'feature1': tf.FixedLenFeature([1], tf.int64)
}

def _parse_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
    parsed_example = tf.parse_single_example(example_proto, feature_description)
    parsed_example["feature0"] = tf.transpose(tf.reshape(parsed_example['feature0'], (256,128)))
    return parsed_example

def create_dataset(filepath):
    
    dataset = tf.data.TFRecordDataset(filepath)
    
    dataset = dataset.map(_parse_function) #, num_parallel_calls=8)
    
    # This dataset will go on forever
    dataset = dataset.repeat()
    
    # Set the number of datapoints you want to load and shuffle 
    dataset = dataset.shuffle(SHUFFLE_BUFFER)
    dataset = dataset.batch(BATCH_SIZE)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    # Create your tf representation of the iterator
    feature = iterator.get_next()
    #print(feature)
    lmfcc = feature["feature0"]
    label = feature["feature1"]
    
    # Bring your picture back in shape
    lmfcc = tf.reshape(lmfcc, [-1,128, 256])
    
    # Create a one hot array for your labels
    label = tf.one_hot(label, NUM_CLASSES)
    print(lmfcc.shape)
    print(label.shape)

    return lmfcc, label

NameError: name 'tf' is not defined

In [4]:
lmfcc, label = create_dataset("../data/debug/sample.tfrecords")

NameError: name 'create_dataset' is not defined

In [4]:
# dir = "../data/debug/"
# ds = load_dataset(dir)
# x_train, y_train = ds.get_train_full()
# x_val, y_val = ds.get_val_full()

batch_size = 32
num_classes = 12

N_LAYERS = 3
FILTER_LENGTH = 5
CONV_FILTER_COUNT = 56
BATCH_SIZE = 32
LSTM_COUNT = 96
EPOCH_COUNT = 2  #70
NUM_HIDDEN = 64
L2_regularization = 0.001


def conv_recurrent_model_build(model_input):
    print('Building model...')
    layer = model_input

    ### 3 1D Convolution Layers
    for i in range(N_LAYERS):
        # give name to the layers
        layer = Conv1D(
            filters=CONV_FILTER_COUNT,
            kernel_size=FILTER_LENGTH,
            kernel_regularizer=regularizers.l2(
                L2_regularization),  # Tried 0.001
            name='convolution_' + str(i + 1))(layer)
        layer = BatchNormalization(momentum=0.9)(layer)
        layer = Activation('relu')(layer)
        layer = MaxPooling1D(2)(layer)
        layer = Dropout(0.4)(layer)

    ## LSTM Layer
    layer = LSTM(LSTM_COUNT, return_sequences=False)(layer)
    layer = Dropout(0.4)(layer)

    ## Dense Layer
    layer = Dense(NUM_HIDDEN,
                  kernel_regularizer=regularizers.l2(L2_regularization),
                  name='dense1')(layer)
    layer = Dropout(0.4)(layer)

    ## Softmax Output
    layer = Dense(num_classes)(layer)
    layer = Activation('softmax', name='output_realtime')(layer)
    model_output = layer
    model = Model(model_input, model_output)

    opt = Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    #print(model.summary())
    return model


def train_model(x_train, y_train, x_val, y_val, checkpoint_name):

    print(x_train.shape)
    n_features = x_train[0].shape[0]
    input_shape = (n_features, 256)
    model_input = Input(tensor=x_train)

    model = conv_recurrent_model_build(model_input)

    tb_callback = TensorBoard(log_dir='./logs/4',
                              histogram_freq=1,
                              batch_size=32,
                              write_graph=True,
                              write_grads=False,
                              write_images=False,
                              embeddings_freq=0,
                              embeddings_layer_names=None,
                              embeddings_metadata=None)

    checkpoint_callback = ModelCheckpoint('../models/' + checkpoint_name +
                                          '{epoch:02d}-{val_loss:.2f}.hdf5',
                                          monitor='val_acc',
                                          verbose=1,
                                          save_best_only=True,
                                          mode='max')

    reducelr_callback = ReduceLROnPlateau(monitor='val_acc',
                                          factor=0.5,
                                          patience=10,
                                          min_delta=0.01,
                                          verbose=1)

    callback_list = [checkpoint_callback, reducelr_callback]

    #     if(os.path.isfile('../models/'+checkpoint_name+'.hdf5')):
    #         print("Weights already exists. Change Name!")
    #         return

    # Fit the model and get training history.
    print('Training...')
    history = model.fit(x_train,
                        y_train,
                        batch_size=BATCH_SIZE,
                        epochs=EPOCH_COUNT,
                        validation_data=(x_val, y_val),
                        callbacks=callback_list,
                        verbose=1)

    return model, history


# Better to change checkpoint name before run
lmfcc, label = create_dataset("../data/debug/sample.tfrecords")
model, history = train_model(lmfcc, label, lmfcc, label, "trail_Run")

# model, history = train_model(np.array(x_train), np.array(y_train),
#                              np.array(x_val), np.array(y_val),
#                             "trail_Run")
print("DONE!!!")

(32, 128, 256)
(32, 1, 41)
(32, 128, 256)
Building model...


AttributeError: Tensor.op is meaningless when eager execution is enabled.